In [22]:
import pandas as pd

import mobile_traffic as mobile
import insee.iris as iris
import election_france as election
from datetime import time
import geopandas as gpd
import numpy as np

## Maps

In [23]:
paris_mobile_data = mobile.get_city_traffic_data(traffic_type=mobile.TrafficType.USERS, city=mobile.City.PARIS)

100%|██████████| 77/77 [01:21<00:00,  1.06s/it]


In [24]:
paris_consumption_by_iris = paris_mobile_data.get_service_consumption_by_location(start=time(hour=19), end=time(hour=23, minute=59))

In [25]:
paris_iris = paris_consumption_by_iris.index

In [26]:
paris_geo = iris.geo.get_geo_data(iris=paris_iris)

In [27]:
paris_facebook_share = paris_consumption_by_iris.div(paris_consumption_by_iris.sum(axis=1), axis=0)[[mobile.Service.FACEBOOK.value, mobile.Service.FACEBOOK_LIVE.value, mobile.Service.TWITTER.value, mobile.Service.YOUTUBE.value]].sum(axis=1).to_frame(name='social_media_share')

In [28]:
extremist_parties = election.Party.get_extremist_party_list_number()
paris_extremist_votes = election.get_percentage_votes_for_party_at_iris_level(list_number=extremist_parties, iris=paris_iris, return_radius=False)
paris_extremist_vote = paris_extremist_votes.sum(axis=1).to_frame(name='extremist_vote')

In [29]:
paris_median_income = iris.admin.get_data(iris=paris_iris, var_name='DEC_MED19')
paris_education = iris.admin.get_data(iris=paris_iris, var_name='P19_ACT_DIPLMIN', shares=True)

In [30]:
paris_income_extremism_facebook = pd.concat([paris_facebook_share, paris_extremist_vote, paris_median_income, paris_education], axis=1)
paris_income_extremism_facebook.columns = ['social_media_share', 'extremist_vote', 'median_income', 'share_high_school_graduates']

In [31]:
paris_map = gpd.GeoDataFrame(paris_income_extremism_facebook, geometry=paris_geo['geometry'].loc[paris_income_extremism_facebook.index])
paris_map.dropna(inplace=True)

## Social Media Share Income and Extremism in France

In [11]:
political_social_media = [mobile.Service.FACEBOOK.value, mobile.Service.FACEBOOK_LIVE.value, mobile.Service.TWITTER.value, mobile.Service.YOUTUBE.value]
social_media_shares = []
for city in mobile.City:
    city_mobile_data = mobile.get_city_traffic_data(traffic_type=mobile.TrafficType.USERS, city=city)
    city_consumption_by_iris = city_mobile_data.get_service_consumption_by_location(start=time(hour=19), end=time(hour=23, minute=59))
    social_media_share = city_consumption_by_iris.div(city_consumption_by_iris.sum(axis=1), axis=0)[political_social_media].sum(axis=1)
    social_media_share_df = social_media_share.to_frame(name='social_media_share').reset_index(names=['iris'])
    social_media_share_df['city'] = city.value
    social_media_shares.append(social_media_share_df)
    
social_media_shares = pd.concat(social_media_shares, axis=0)
social_media_shares.set_index('iris', inplace=True)

100%|██████████| 77/77 [00:13<00:00,  5.66it/s]


In [12]:
social_media_shares_no_dup = social_media_shares.reset_index().drop_duplicates(subset=['iris']).set_index('iris')[['social_media_share']]

In [13]:
iris_social_media_share_data = social_media_shares_no_dup.index

In [14]:
income = iris.admin.get_data(iris=iris_social_media_share_data, var_name='DEC_MED19')

In [15]:
extremist_vote = election.get_percentage_votes_for_party_at_iris_level(list_number=extremist_parties, iris=iris_social_media_share_data)
extremist_vote = extremist_vote.loc[(extremist_vote['radius'] < 1200)]
extremist_vote = extremist_vote.drop(columns=['radius'])
extremism = extremist_vote.sum(axis=1).to_frame(name='extremist_vote')

In [16]:
social_media_shares_income_and_extremism = pd.concat([social_media_shares_no_dup, income, extremism], axis=1)
social_media_shares_income_and_extremism.columns = ['social_media_share', 'median_income', 'extremist_vote']

In [17]:
social_media_shares_income_and_extremism.dropna(inplace=True)

## Regression of Social Media on Extremism with controls 

In [18]:
age_vars = ['P19_POP1529', 'P19_POP3044', 'P19_POP4559', 'P19_POP6074', 'P19_POP75P']
education_vars = ['P19_ACT_DIPLMIN','P19_ACT_BAC', 'P19_ACT_SUP2']
unemployment_vars = ['P19_CHOM1524', 'P19_CHOM2554', 'P19_CHOM5564']
immigration_vars = ['P19_POP_IMM']
gender_vars = ['P19_POPH'] 
income_vars = ['DEC_MED19']

In [19]:
age = iris.admin.get_data(iris=iris_social_media_share_data, var_name=age_vars, shares=True)
education = iris.admin.get_data(iris=iris_social_media_share_data, var_name=education_vars, shares=True)
unemployment = iris.admin.get_data(iris=iris_social_media_share_data, var_name=unemployment_vars, shares=True)
immigration = iris.admin.get_data(iris=iris_social_media_share_data, var_name=immigration_vars, shares=True)
gender = iris.admin.get_data(iris=iris_social_media_share_data, var_name=gender_vars, shares=True)
income = iris.admin.get_data(iris=iris_social_media_share_data, var_name=income_vars, shares=False)
controls = pd.concat([age, education, unemployment, immigration, gender, income], axis=1)

In [20]:
social_media_shares_extremism_and_controls = pd.concat([social_media_shares_no_dup, extremism, controls], axis=1)

In [21]:
social_media_shares_extremism_and_controls.dropna(inplace=True)

## Save

In [32]:
path = '/Users/anmusso/Desktop/PhD/Projects/Current/NetMob/NetMobData/data/FigureData/GeoDiscontent/'

In [33]:
paris_map.to_file(path + 'paris_map.geojson', driver='GeoJSON')

In [37]:
social_media_shares_income_and_extremism.to_csv(path + 'social_media_shares_income_and_extremism.csv')

In [38]:
social_media_shares_extremism_and_controls.to_csv(path + 'social_media_shares_extremism_and_controls.csv')

In [34]:
import pandas as pd
import geopandas as gpd

In [42]:
paris_map = gpd.read_file(path + 'paris_map.geojson')
social_media_shares_income_and_extremism = pd.read_csv(path + 'social_media_shares_income_and_extremism.csv', index_col='iris')
social_media_shares_extremism_and_controls = pd.read_csv(path + 'social_media_shares_extremism_and_controls.csv', index_col='iris')

In [43]:
paris_map.columns

Index(['iris', 'social_media_share', 'extremist_vote', 'median_income',
       'share_high_school_graduates', 'geometry'],
      dtype='object')

In [44]:
paris_map.to_crs(epsg=2154, inplace=True)
area_ = paris_map.area.mean()

In [45]:
area_ / (1000 * 1000)

0.27776135053485707

In [46]:
m = paris_map['social_media_share'].mean()
m

0.1973663114730219

In [48]:
s = paris_map['social_media_share'].std()
s

0.016134071974064138

In [49]:
m + s

0.21350038344708602

In [1]:
import insee

In [41]:
geo_iris = insee.iris.geo.get_geo_data(iris=paris_map.index)

KeyError: 'None of [RangeIndex(start=0, stop=2711, step=1)] are in the [index]'

In [3]:
0.277

0.277

In [4]:
100*100 / (1000* 1000)

0.01

In [6]:
0.277 / 0.01

27.700000000000003